In [5]:
from math import *
import numpy as np

In [6]:
def eval_func(t):
    '''
    Интегрируемая функция
    '''
    def inner_func_LR(phi_x, theta_y):
        return (2*cos(theta_y)) / (1-(sin(theta_y)**2)*(cos(phi_x)**2))
    def inner_func_main(t, LR_func, phi_x, theta_y):
        return (1-exp((-t)*LR_func(phi_x, theta_y))) * cos(theta_y)*sin(theta_y)
    return lambda phi_x, theta_y: inner_func_main(t, inner_func_LR, phi_x, theta_y)

In [7]:
def eval_Simpson_multiple_integration(func, a, b, c, d, N_nodes, M_nodes):
    h = (b-a)/N_nodes
    res = 0
    eval_func = lambda x: eval_Gauss_integration(func, c, d, M_nodes, x)
    for i in range(0, int((N_nodes/2))-1, 1):
        res += (
            eval_func(a+2*i*h) + 4*eval_func(a+(2*i+1)*h) + eval_func(a+(2*i+2)*h)
        )
        #print("\ti=", i, "res=", res)
    return h/3 * res

def eval_Simpson_multiple_integration2(func, a, b, c, d, N_nodes, M_nodes):
    h = (b-a)/N_nodes
    res = 0
    x_start = a
    eval_func = lambda x: eval_Gauss_integration(func, c, d, M_nodes, x)
    while x_start < b:
        next_x = x_start + h
        res += (
            eval_func(x_start) + 4*eval_func(x_start+h/2) + eval_func(next_x)
        )
        x_start = next_x
    return h/6 * res

def eval_Gauss_integration(func, c, d, M_nodes, x):
    '''
    Проводим численное интегрирование по формуле Гаусса
    '''
    T = eval_Lm_roots(M_nodes)
    A = NP_MATRIX_EVAL(M_nodes, T)
    sum_Aiti = sum([Ai*func(x, (c+d)/2+((d-c)/2)*ti) for (Ai, ti) in zip(A, T)])
    res = ((d-c)/2)*sum_Aiti
    return res

def NP_MATRIX_EVAL(n, Lm_roots):
    '''
    Решаем СЛАУ: выичисляем коэфициенты Ai
    '''
    A = np.array([[ti ** k for ti in Lm_roots] for k in range(n)])
    B = np.array([eval_At_sum(k) for k in range(n)])
    res = np.linalg.solve(A, B)
    return res

def eval_At_sum(k):
    '''
    Вычисление суммы произведений Ai*ti
    '''
    return 2/(k+1) if k%2==0 else 0  

In [8]:
def machine_epsilon(func=float):
    '''
    Вспомогательная функция для вычисления машинного нуля
    '''
    machine_epsilon = func(1)
    while func(1)+func(machine_epsilon) != func(1):
        machine_epsilon_last = machine_epsilon
        machine_epsilon = func(machine_epsilon) / func(2)
    return machine_epsilon_last

def eval_Lm(x, n):
    '''
    Вычисление полинома Лежандра n-ой степени
    '''
    res = None
    if n > 1:
        Lm0 = 1 #значение полинома 0-й степени
        Lm1 = x #значение полинома 1-й степени
        lim_n = 2
        while True:
            res = (
                ((2*lim_n-1)*x*Lm1 - (lim_n-1)*Lm0)
                /lim_n
            )
            Lm0 = Lm1
            Lm1 = res
            if lim_n == n:
                break
            lim_n += 1
        return res
    else:
        return 1 if n == 0 else x

def eval_Lm_derivative(x, n):
    '''
    Вычисление производной полинома Лежандра
    '''
    L_first = eval_Lm(x, n-1)
    L_second = eval_Lm(x, n)
    return (n/(1-x**2))*(L_first - x*L_second)

def eval_Lm_roots(n):
    '''
    Вычисление заданного количества корней полинома Лежандра
    '''
    ME = machine_epsilon()*10
    roots_res = []
    roots_init_approx = [
        cos(
            (pi*(4*i-1)) / (4*n+2)
        )
        for i in range(1, n+1)
    ]
    for x in roots_init_approx:
        Lm = 1
        while Lm > ME:
            Lm = eval_Lm(x, n)
            Lm_deriv = eval_Lm_derivative(x, n)
            x = x - Lm/Lm_deriv
        roots_res.append(x)
    return roots_res

print(eval_Lm_roots(16))

[0.9894009349916499, 0.944576551633332, 0.8656312023878318, 0.7554046094313628, 0.6178762444026438, 0.4580168021068179, 0.2816035507792589, 0.09501251001184953, -0.09501251001184953, -0.2816035507792589, -0.4580168021068179, -0.6178762444026438, -0.7554046094313629, -0.8656312023878318, -0.944576551633332, -0.9894009349916499]


In [9]:
def integrate(N, M, t):
    ef = eval_func(t)
    res = eval_Simpson_multiple_integration(
        ef,
        0, pi/2,
        0, pi/2,
        N, M
    )
    return (4/pi) * res

print(integrate(4, 5, 0.9))
t = 0.05
for i in range(21):
    print("t=", t, integrate(4,5,t))
    t += 0.5

0.43367940876864514
t= 0.05 0.05855556726822834
t= 0.55 0.36001479194632235
t= 1.05 0.4511952887807398
t= 1.55 0.48124057358559563
t= 2.05 0.4918707346011229
t= 2.55 0.4959551093209039
t= 3.05 0.4976872397539759
t= 3.55 0.49850837037383383
t= 4.05 0.49894501913326883
t= 4.55 0.49920360529032265
t= 5.05 0.4993716340665837
t= 5.55 0.49948933357465836
t= 6.05 0.4995767101466126
t= 6.55 0.49964446128859963
t= 7.05 0.49969869411670853
t= 7.55 0.4997431094924231
t= 8.05 0.4997800779926997
t= 8.55 0.4998111991653991
t= 9.05 0.4998376055682377
t= 9.55 0.49986013447600647
t= 10.05 0.4998794282049617
